In [1]:
cd /home/jovyan

/home/jovyan


In [2]:
import pickle
from pymongo import MongoClient

mongo_client = MongoClient('this_mongo')
data_collection = mongo_client.data.datasets

target_dict_from_mongo = data_collection.find_one({"dataset" : "target"})
target_from_mongo = pickle.loads(target_dict_from_mongo["data"])
features_dict_from_mongo = data_collection.find_one({"dataset" : "features"})
features_from_mongo = pickle.loads(features_dict_from_mongo["data"])
final_features_names_dict_from_mongo = data_collection.find_one({"dataset" : "final_features_names"})
final_features_names_from_mongo = final_features_names_dict_from_mongo["data"]

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
rfc = RandomForestClassifier(n_estimators=30)
grid_search = GridSearchCV(rfc, {}, cv=5, return_train_score=False)
grid_search.fit(features_from_mongo[final_features_names_from_mongo], target_from_mongo)
best_estimator = grid_search.best_estimator_
grid_search.best_score_

0.81904382998474901

In [5]:
marital_status_Married = [0, 1]
relationship_Husband   = [0, 1]
marital_status_Never   = [0, 1]
education_Bachelors    = [0, 1]
occupation_Exec        = [0, 1]
occupation_Prof        = [0, 1]
gender                 = [0, 1]
education_Masters      = [0, 1]

features_for_lookup = [ 
    [val_1, val_2, val_3, val_4, val_5, val_6, val_7, val_8]
        for val_1 in marital_status_Married
        for val_2 in relationship_Husband
        for val_3 in marital_status_Never
        for val_4 in education_Bachelors
        for val_5 in occupation_Exec
        for val_6 in occupation_Prof
        for val_7 in gender
        for val_8 in education_Masters
]

In [6]:
import numpy as np

features_for_lookup_np = np.array(features_for_lookup)
features_for_lookup_np.shape

(256, 8)

In [7]:
predictions = best_estimator.predict(features_for_lookup_np)

In [8]:
marital_status_Married = ['0', '1']
relationship_Husband   = ['0', '1']
marital_status_Never   = ['0', '1']
education_Bachelors    = ['0', '1']
occupation_Exec        = ['0', '1']
occupation_Prof        = ['0', '1']
gender                 = ['0', '1']
education_Masters      = ['0', '1']

features_as_keys = [ 
    ''.join([val_1, val_2, val_3, val_4, val_5, val_6, val_7, val_8])
        for val_1 in marital_status_Married
        for val_2 in relationship_Husband
        for val_3 in marital_status_Never
        for val_4 in education_Bachelors
        for val_5 in occupation_Exec
        for val_6 in occupation_Prof
        for val_7 in gender
        for val_8 in education_Masters
]

In [9]:
lookup_table = [
    {"feature_set" : feature_set,
     "prediction" : prediction }
    for feature_set, prediction in zip(features_as_keys, predictions.astype(str))
]

In [10]:
import pickle
from pymongo import MongoClient

mongo_client = MongoClient('this_mongo')
predictions_collection = mongo_client.data.predictions

In [11]:
predictions_collection.drop()

In [12]:
predictions_collection.insert_many(lookup_table)

In [13]:
%run src/prepare_lookup_table.py

In [14]:
predictions_collection.count_documents({})

256

In [15]:
predictions_collection.find_one({"feature_set" : "01010101"})

{'_id': ObjectId('5bd9e76e615d0701a37d1d30'),
 'feature_set': '01010101',
 'prediction': '1'}